## One-unit neural network model for logistic regression

In this notebook, we illustrate how to do [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression)
using a neural network-like implementation with one single unit (perceptron).
This is equivalent to a logistic regression model, only it is solved using neural networks (forward and backpropagation) rather than with maximum likelihood (or similar algorithms).

As example data, we use the famous [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set)

Basically, a neural network implementation of logistic regression will look like the sketch below: first, the vector of input features $\mathbf{x}$ is multiplied by the vector of weights $\mathbf{w}$, the results are summed up together and the bias term $b$ is added. This will return the real-valued variable $z$ in the interval $[\pm ∞]$.
Then, $z$ is activated with the logistic (sigmoid) function $\rightarrow \sigma(z)$ to give $P(y=1|x)$, the probability of belonging to class `1` given the input features $x$.

<img src="logistic_perceptron.png" alt="perceptron" style="width: 500px;"/>

## Loading libraries and setting the random seed

First of all, we load some necessary libraries; then we setup the random seed to ensure reproducibility of results. Since tensorflow uses an internal random generator we need to fix both the general seed (via numpy `seed()`) and tensorflow seed (via `set_seet()`)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

  # Set the seed using keras.utils.set_random_seed. This will set:
  # 1) `numpy` seed
  # 2) `tensorflow` random seed
  # 3) `python` random seed
tf.keras.utils.set_random_seed(10)

  # This will make TensorFlow ops as deterministic as possible, but it will
  # affect the overall performance, so it's not enabled by default.
  # `enable_op_determinism()` is introduced in TensorFlow 2.9.
tf.config.experimental.enable_op_determinism()

## Get the data

From `sklearn.datasets` there are generally two ways to import the data:

1. `return_X_y = False` (default): returns a "bunch" object that contains both the `target` and the `features` (to be accessed as attributes): `<dataset>.target` and `<dataset>.data`
2. `return_X_y = True`: returns directly the target and features separately

In [ ]:
import sklearn.datasets

## 1)
iris = sklearn.datasets.load_iris()
features = iris.data
target = iris.target

In [ ]:
## 2)
(features_, target_) = sklearn.datasets.load_iris(return_X_y = True) ## feature names are not returned

In [ ]:
target == target_

Next, we convert data and features (originally as `numpy` arrays) to `pandas` dataframes / series

In [ ]:
iris.data = pd.DataFrame(features, columns=iris.feature_names) #converting numpy array -> pandas DataFrame
iris.target = pd.Series(target) #converting numpy array -> pandas Series

The **iris** dataset is a historic dataset first used by **Ronald Fisher** in his 1936 paper on linear discriminant analysis (LDA).
The dataset contains information on 150 flower samples, belonging to three species of Iris (Iris setosa, Iris virginica and Iris versicolor: the `target`).
Four `features` were measured from each sample: the length and the width of the sepals and of the petals, in centimeters.

In [ ]:
print(iris.DESCR)

## Exploratory Data Analysis (EDA)

As said, the feature data has 150 rown and 4 columns:

In [ ]:
print('Shape of the feature table: ' + str(iris.data.shape))

The four features are real-valued numbers in the $10^1$ range:

In [ ]:
iris.data.describe()

The `target` is a vector (`pandas` series) of integers (0, 1, 2), one for each flower species (50 samples each):

In [ ]:
#using Counter object to print a tally of the classes
from collections import Counter
print('Numerosity for each class: ' + str(Counter(iris.target)))

Classes are represented via a numeric index: 0 for *setosa*, 1 for *versicolor*, 2 for *virginica*. The samples are in order: the first 50 samples are *setosa*, then 50 *versicolor* and the last 50 are *virginica*.

When working with a new dataset, it is always importat to plot the data. We are unfortunately talking about a 5-dimensional dataset (the four features + the target class) which is not easily representable.
One possibility is to take a slice (a subset) of the whole dataset.

In the next code chunk we plot two features plus the class.

In [ ]:
## SELECT FEATURES TO PLOT
#change these two values to plot different features, remembering the numbering:
# 0 : sepal length (cm)
# 1 : sepal width (cm)
# 2 : petal length (cm)
# 3 : petal width (cm)
feature_x = 0
feature_y = 1

#starting a new plot
fig, ax = plt.subplots()

#adding data in three bunches of 50, once per class
ax.scatter(x=iris.data.iloc[0:50,feature_x],    y=iris.data.iloc[0:50,feature_y],    c='red',   label=iris.target_names[0])
ax.scatter(x=iris.data.iloc[50:100,feature_x],  y=iris.data.iloc[50:100,feature_y],  c='green', label=iris.target_names[1])
ax.scatter(x=iris.data.iloc[100:150,feature_x], y=iris.data.iloc[100:150,feature_y], c='blue',  label=iris.target_names[2])

#the axis names are taken from feature names
ax.set_xlabel(iris.feature_names[feature_x])
ax.set_ylabel(iris.feature_names[feature_y])

#adding the legend and printing the plot
ax.legend()
plt.show()

The plot shows clearly that setosa is quite separate from the other two classes. Whichever features you choose for the plot, you more or less get this same general result.

This is however a **three-class** problem: however, we want to build a **neural network model** for **logistic regression** applied to a **binary classification problem**.

Therefore, we need some preparatory ('preprocessing') steps.

## Data preprocessing

To make things a little more interesting we decide to **renounce to half of our features**, **using only the first two** columns. Moreover, we **join together Setosa and Versicolor**. In other words, we want a classifier able to discriminate virginica (which becomes the new class "1") from the other irises (which all together become the new class "0"):